In [1]:
!conda install -c conda-forge numpy swig boost-cpp libboost sphinx sphinx_rtd_theme -y
!pip install vina
!conda install -c conda-forge mdanalysis -y
!pip install rdkit
!pip install prolif
!conda install nglview -c conda-forge -y
!pip install pdb2pqr
# !pip install meeko
!pip install biopython
!pip install gemmi
!pip install py3Dmol
!pip install --prefer-binary pyscf
!pip install qiskit==1.4.3
!pip install qiskit-nature

                                                                                
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [1]:
import os
import requests
import numpy as np
import pandas as pd
import warnings
from Bio.PDB import PDBList, PDBParser, Select, PDBIO
import MDAnalysis as mda
import nglview as nv
from rdkit import Chem
from rdkit.Chem import AllChem, rdmolfiles
from vina import Vina
import prolif as plf
from meeko import PDBQTMolecule, RDKitMolCreate
from pyscf import gto, scf, dft
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo

/home/kyle/miniforge3/envs/vina_vqe/lib/python3.13/site-packages/nglview/__init__.py:12: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
/home/kyle/miniforge3/envs/vina_vqe/lib/python3.13/site-packages/MDAnalysis/topology/tables.py:52: DeprecationWarning: Deprecated in version 2.8.0
MDAnalysis.topology.tables has been moved to MDAnalysis.guesser.tables. This import point will be removed in MDAnalysis version 3.0.0
  warnings.warn(wmsg, category=DeprecationWarning)


In [2]:
protein_folder = 'protein/'
ligand_folder = 'ligand/'

os.makedirs(protein_folder, exist_ok=True)
os.makedirs(ligand_folder, exist_ok=True)

## Protein Preparation

In [3]:
protein_id = "2BMF"

pdb_request = requests.get(f"https://files.rcsb.org/download/{protein_id}.pdb")
if pdb_request.status_code == 200:
    with open(f"{protein_folder}/{protein_id}.pdb", "w+") as f:
        f.write(pdb_request.text)
else: raise Exception("Fetch error")

In [4]:
u = mda.Universe(f"{protein_folder}{protein_id}.pdb")
nv.show_mdanalysis(u)

/home/kyle/miniforge3/envs/vina_vqe/lib/python3.13/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/kyle/miniforge3/envs/vina_vqe/lib/python3.13/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


NGLWidget()

In [5]:
protein = u.select_atoms("protein and segid A")
water = u.select_atoms("resname HOH and segid A")
view = nv.show_mdanalysis(protein)
view.clear_representations()
view.add_representation('surface', colorScheme="hydrophobicity")
water_view = nv.show_mdanalysis(water)
water_view.add_representation('spacefill')
view

/home/kyle/miniforge3/envs/vina_vqe/lib/python3.13/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/kyle/miniforge3/envs/vina_vqe/lib/python3.13/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


NGLWidget()

In [6]:
protein.write(f"{protein_folder}protein_{protein_id}.pdb")

/home/kyle/miniforge3/envs/vina_vqe/lib/python3.13/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"


In [7]:
!pdb2pqr --pdb-output="{protein_folder}protein_h.pdb" --pH=7.4 "{protein_folder}protein_{protein_id}.pdb" "{protein_folder}protein_{protein_id}.pqr" --whitespace

/home/kyle/miniforge3/envs/vina_vqe/lib/python3.13/pty.py:95: DeprecationWarning: This process (pid=16845) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


INFO:PDB2PQR v3.7.1: biomolecular structure conversion software.
INFO:Please cite:  Jurrus E, et al.  Improvements to the APBS biomolecular solvation software suite.  Protein Sci 27 112-128 (2018).
INFO:Please cite:  Dolinsky TJ, et al.  PDB2PQR: expanding and upgrading automated preparation of biomolecular structures for molecular simulations. Nucleic Acids Res 35 W522-W525 (2007).
INFO:Checking and transforming input arguments.
INFO:Loading topology files.
INFO:Loading molecule: protein/protein_2BMF.pdb
ERROR:Error parsing line: invalid literal for int() with base 10: ''
ERROR:<REMARK     2>
ERROR:Truncating remaining errors for record type:REMARK

ERROR:['REMARK']
INFO:Setting up molecule.
INFO:Created biomolecule object with 442 residues and 3531 atoms.
INFO:Setting termini states for biomolecule chains.
INFO:Loading forcefield.
INFO:Loading hydrogen topology definitions.
INFO:Attempting to repair 24 missing atoms in biomolecule.
INFO:Added atom CG to residue LYS A 199 at coordinat

In [8]:
u = mda.Universe(f"{protein_folder}protein_{protein_id}.pqr")
u.atoms.write(f"{protein_folder}{protein_id}.pdbqt")

/home/kyle/miniforge3/envs/vina_vqe/lib/python3.13/site-packages/MDAnalysis/coordinates/PDBQT.py:305: UserWarning: Supplied AtomGroup was missing the following attributes: altLocs, occupancies, tempfactors. These will be written with default values. 
  warnings.warn(


In [9]:
# Read in the just-written PDBQT file, replace text, and write back
with open(f"{protein_folder}{protein_id}.pdbqt", 'r') as file:
    file_content = file.read()

# Replace 'TITLE' and 'CRYST1' with 'REMARK'
file_content = file_content.replace('TITLE', 'REMARK').replace('CRYST1', 'REMARK')

# Write the modified content back to the file
with open(f"{protein_folder}{protein_id}.pdbqt", 'w') as file:
    file.write(file_content)

## Ligand Preparation

In [10]:
smiles = "CCCSc1ncc(c(n1)C(=O)Nc2nc3ccc(cc3s2)OC)Cl"

mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)
AllChem.EmbedMolecule(mol)
AllChem.UFFOptimizeMolecule(mol)

writer = rdmolfiles.PDBWriter(f"{ligand_folder}ligand.pdb")
writer.write(mol)
writer.close()

Chem.MolToMolFile(mol, f"{ligand_folder}ligand.mol")

In [11]:
u = mda.Universe(f"{ligand_folder}ligand.pdb")
nv.show_mdanalysis(u)

/home/kyle/miniforge3/envs/vina_vqe/lib/python3.13/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


NGLWidget()

In [12]:
# Use meeko to prepare small molecules - using meeko helps us visualize them later.
!mk_prepare_ligand.py -i "{ligand_folder}ligand.mol" -o "{ligand_folder}prepared_ligand.pdbqt"

/home/kyle/miniforge3/envs/vina_vqe/lib/python3.13/pty.py:95: DeprecationWarning: This process (pid=16845) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


Input molecules processed: 1, skipped: 0
PDBQT files written: 1
PDBQT files not written due to error: 0
Input molecules with errors: 0


## Pre-Docking: Defining the Search Box

In [13]:
u = mda.Universe(f"{protein_folder}protein_{protein_id}.pdb")
res1 = u.select_atoms("resid 388")
nv.show_mdanalysis(res1)

/home/kyle/miniforge3/envs/vina_vqe/lib/python3.13/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


NGLWidget()

In [14]:
res2 = u.select_atoms("resid 599")
nv.show_mdanalysis(res2)

/home/kyle/miniforge3/envs/vina_vqe/lib/python3.13/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


NGLWidget()

In [15]:
cg1 = res1.center_of_geometry()
cg2 = res2.center_of_geometry()
coors = np.vstack([cg1, cg2])
center = coors.sum(axis=0) / 2
center = center.tolist()
ligand_box = coors.max(axis=0) - coors.min(axis=0) + 10 # padding of 10
ligand_box = ligand_box.tolist()
print("center:", center)
print("box:", ligand_box)

center: [-4.630055514756929, -3.6067424138086013, 55.17953026896775]
box: [13.110111062160946, 20.84615146180596, 10.402394651162503]


## Docking Ligands with AutoDock Vina

In [16]:
os.makedirs("docking_results", exist_ok=True)

v = Vina(sf_name="vina", seed=1695247494) # seed is set for consistency

In [17]:
v.set_receptor(f"{protein_folder}{protein_id}.pdbqt")
v.set_ligand_from_file(f"{ligand_folder}prepared_ligand.pdbqt")
v.compute_vina_maps(center=center, box_size=ligand_box)

Computing Vina grid ... done.


In [18]:
v.dock(exhaustiveness=32, n_poses=10)

Performing docking (random seed: 1695247494) ... 
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************


In [19]:
v.write_poses(f"docking_results/ligand-{protein_id}.pdbqt", n_poses=10, overwrite=True)

In [20]:
column_names = ["total", "inter", "intra", "torsions", "intra best pose"]
df = pd.DataFrame(v.energies(), columns=column_names)
df.to_csv(f"docking_results/ligand-{protein_id}-energies.csv", index=False)
df

,total,inter,intra,torsions,intra best pose
0,-5.817,-7.857,-0.531,2.040,-0.531
1,-5.611,-7.785,-0.325,1.968,-0.531
2,-5.488,-7.354,-0.590,1.925,-0.531
3,-5.451,-7.811,-0.083,1.912,-0.531
4,-5.434,-7.391,-0.481,1.906,-0.531
5,-5.311,-7.467,-0.238,1.863,-0.531
6,-5.301,-7.545,-0.146,1.859,-0.531
7,-5.286,-7.027,-0.644,1.854,-0.531
8,-5.282,-7.142,-0.523,1.853,-0.531


## Visualizing Docking Results

In [21]:
!mk_export.py "docking_results/ligand-{protein_id}.pdbqt" -s "docking_results/ligand-{protein_id}.sdf"

/home/kyle/miniforge3/envs/vina_vqe/lib/python3.13/pty.py:95: DeprecationWarning: This process (pid=16845) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


In [22]:
protein = rdmolfiles.MolFromPDBFile(f"{protein_folder}protein_h.pdb")

In [23]:
protein_plf = plf.Molecule.from_rdkit(protein)
poses_plf = plf.sdf_supplier(f"docking_results/ligand-{protein_id}.sdf")

In [24]:
fp = plf.Fingerprint(count=True)

In [25]:
# run on your poses
fp.run_from_iterable(poses_plf, protein_plf)

/home/kyle/miniforge3/envs/vina_vqe/lib/python3.13/site-packages/multiprocess/popen_fork.py:67: DeprecationWarning: This process (pid=16845) is multi-threaded, use of fork() may lead to deadlocks in the child.
  self.pid = os.fork()
/home/kyle/miniforge3/envs/vina_vqe/lib/python3.13/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/kyle/miniforge3/envs/vina_vqe/lib/python3.13/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


  0%|          | 0/10 [00:00<?, ?it/s]

<prolif.fingerprint.Fingerprint: 9 interactions: ['Hydrophobic', 'HBAcceptor', 'HBDonor', 'Cationic', 'Anionic', 'CationPi', 'PiCation', 'PiStacking', 'VdWContact'] at 0x788ea9784c20>

In [26]:
pose_index=1
fp.plot_lignetwork(poses_plf[pose_index])

In [27]:
view = fp.plot_3d(
    poses_plf[pose_index], protein_plf, frame=pose_index, display_all=False
)
view

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## Selecting the Binding Interaction

In [28]:
fp_df = fp.to_dataframe()
interaction_dict = dict(fp_df.iloc[0])
interaction_dict

{('UNL1', 'THR224.A', 'VdWContact'): np.uint8(0),
 ('UNL1', 'ARG225.A', 'VdWContact'): np.uint8(0),
 ('UNL1', 'PHE288.A', 'Hydrophobic'): np.uint8(4),
 ('UNL1', 'PHE288.A', 'VdWContact'): np.uint8(1),
 ('UNL1', 'ARG387.A', 'PiCation'): np.uint8(0),
 ('UNL1', 'ARG387.A', 'VdWContact'): np.uint8(0),
 ('UNL1', 'LYS388.A', 'PiCation'): np.uint8(0),
 ('UNL1', 'LYS388.A', 'VdWContact'): np.uint8(1),
 ('UNL1', 'PHE390.A', 'Hydrophobic'): np.uint8(0),
 ('UNL1', 'PHE390.A', 'VdWContact'): np.uint8(1),
 ('UNL1', 'ASP391.A', 'VdWContact'): np.uint8(0),
 ('UNL1', 'ILE410.A', 'Hydrophobic'): np.uint8(2),
 ('UNL1', 'ILE410.A', 'VdWContact'): np.uint8(0),
 ('UNL1', 'LEU443.A', 'Hydrophobic'): np.uint8(0),
 ('UNL1', 'LEU443.A', 'VdWContact'): np.uint8(0),
 ('UNL1', 'ASP541.A', 'VdWContact'): np.uint8(3),
 ('UNL1', 'PRO543.A', 'VdWContact'): np.uint8(0),
 ('UNL1', 'ARG599.A', 'VdWContact'): np.uint8(0),
 ('UNL1', 'ILE600.A', 'Hydrophobic'): np.uint8(0),
 ('UNL1', 'ILE600.A', 'VdWContact'): np.uint8(0),

In [29]:
selected_residue = 'ASP541.A'

selected_key = None
for key in interaction_dict:
    if selected_residue == key[1]:
        selected_key = key
        break

if selected_key == None: raise Exception("Selected residue does not exist.")
if interaction_dict[selected_key] == 0: raise Exception("Selected entry not occuring between the ligand and the protein.")
print(f"{selected_key}: {interaction_dict[selected_key]}")

('UNL1', 'ASP541.A', 'VdWContact'): 3


In [30]:
protein_pdb_file = f"{protein_folder}protein_{protein_id}.pdb"
ligand_pdbqt_file = f"docking_results/ligand-{protein_id}.pdbqt" 

pdbqt_mol_reader = PDBQTMolecule.from_file(ligand_pdbqt_file)
rdkit_mols_from_pdbqt = RDKitMolCreate.from_pdbqt_mol(pdbqt_mol_reader)

if rdkit_mols_from_pdbqt:
    best_pose_mol_rdkit = rdkit_mols_from_pdbqt[0]
    print(f"Successfully loaded the first pose as an RDKit molecule: {Chem.MolToSmiles(best_pose_mol_rdkit)}")

    # Define the output path for the single pose PDB file
    single_pose_ligand_pdb_file = "docking_results/pose.pdb"

    # Save the RDKit molecule of the first pose to a new PDB file
    # Note: RDKit's Chem.MolToPDBFile is typically used for saving RDKit molecules to PDB.
    Chem.MolToPDBFile(best_pose_mol_rdkit, single_pose_ligand_pdb_file)

    print(f"MODEL 1 of the ligand extracted and saved to: {single_pose_ligand_pdb_file}")

    # --- Step 2: Load into MDAnalysis and combine with protein ---
    # Load the protein
    protein_universe = mda.Universe(protein_pdb_file)

    # Load the single ligand pose extracted by Meeko
    ligand_universe = mda.Universe(single_pose_ligand_pdb_file)

    # Merge the protein atoms and the ligand atoms into a new Universe
    combined_universe = mda.Merge(protein_universe.atoms, ligand_universe.atoms)

    print("\nCombined Universe created successfully. It contains:")
    print(f"- Number of protein atoms: {protein_universe.atoms.n_atoms}")
    print(f"- Number of ligand atoms (from MODEL 1): {ligand_universe.atoms.n_atoms}")
    print(f"- Total atoms in combined universe: {combined_universe.atoms.n_atoms}")

else:
    raise Exception("Could not extract any poses from the PDBQT file using Meeko.")

Successfully loaded the first pose as an RDKit molecule: [H]c1nc(SC([H])([H])C([H])([H])C([H])([H])[H])nc(C(=O)N([H])c2nc3c([H])c([H])c(OC([H])([H])[H])c([H])c3s2)c1Cl
MODEL 1 of the ligand extracted and saved to: docking_results/pose.pdb

Combined Universe created successfully. It contains:
- Number of protein atoms: 3531
- Number of ligand atoms (from MODEL 1): 40
- Total atoms in combined universe: 3571


In [31]:
nv.show_mdanalysis(combined_universe)

/home/kyle/miniforge3/envs/vina_vqe/lib/python3.13/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/kyle/miniforge3/envs/vina_vqe/lib/python3.13/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


NGLWidget()

In [32]:
selected_residue_id = "541"
selected_interaction = combined_universe.select_atoms(f"resid {selected_residue_id} or resname UNL")
nv.show_mdanalysis(selected_interaction)

/home/kyle/miniforge3/envs/vina_vqe/lib/python3.13/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


NGLWidget()

## Define the Molecules for Quantum Level Interaction

In [33]:
mol_id = "H6"
center_mol = selected_interaction.select_atoms(f"resname UNL and resid 1 and name {mol_id}")
center_coor = center_mol.positions[0]
d = np.linalg.norm(selected_interaction.positions - center_coor, axis=1)

In [34]:
radius = 2.5 # define the radius within the center

molecules = []

for i in range(len(d)):
    if d[i] <= radius:
        id_ = int(selected_interaction.ids[i])
        element = selected_interaction.elements[i]
        coors = selected_interaction.positions[i]
        molecules.append([id_, element, coors])
        print(f"{id_}\t{element}{"*" if all(coors == center_coor) else ""}\t{d[i]:.2f}\t{np.round(coors,2)}")

print(f"Atom count: {len(molecules)}")

2879	O	2.50	[-4.81 -5.13 48.29]
2	C	2.16	[-6.79 -1.78 50.04]
3	C	1.10	[-5.65 -2.74 50.41]
4	S	2.42	[-6.3  -4.29 51.1 ]
30	H	2.26	[-6.92 -1.8  48.94]
31	H*	0.00	[-5.06 -2.96 49.5 ]
32	H	1.81	[-5.02 -2.25 51.16]
Atom count: 7


## Quantum-Level Interaction Computation

In [35]:
atom_string = ""
atomic_numbers = {
    'H': 1, 'He': 2, 'Li': 3, 'Be': 4, 'B': 5, 'C': 6, 'N': 7, 'O': 8, 'F': 9,
    'Ne': 10, 'Na': 11, 'Mg': 12, 'Al': 13, 'Si': 14, 'P': 15, 'S': 16, 'Cl': 17,
    'Ar': 18, # ... add more as needed
}

total_protons = 0
for atom_id, symbol, coords in molecules:
    atom_string += f"{symbol} {coords[0]:.3f} {coords[1]:.3f} {coords[2]:.3f}\n"
    total_protons += atomic_numbers.get(symbol, 0) # Get atomic number, default to 0 if not found

print(atom_string)

O -4.812 -5.132 48.288
C -6.791 -1.781 50.036
C -5.652 -2.735 50.407
S -6.305 -4.291 51.105
H -6.923 -1.804 48.944
H -5.065 -2.963 49.505
H -5.019 -2.249 51.164



### Classical Computing Benchmark

In [36]:
# --- 2. Determine Charge and Spin Correctly ---

# Initial assumptions for the cluster model:
# Assume the cluster model itself is neutral (charge = 0)
# This is usually the case unless you explicitly select ions.
desired_molecular_charge = 0

# Calculate the actual number of electrons based on the sum of atomic numbers and the desired charge
# For a neutral molecule, nelectrons = total_protons
# For a +1 cation, nelectrons = total_protons - 1
# For a -1 anion, nelectrons = total_protons + 1
num_electrons = total_protons - desired_molecular_charge

# Determine spin based on num_electrons
# If num_electrons is even, it's typically a singlet (spin=0) - closed shell
# If num_electrons is odd, it's typically a doublet (spin=1) - open shell
if num_electrons % 2 == 0:
    mol_spin = 0  # Even number of electrons -> Singlet (2S = 0)
    print(f"Calculated {num_electrons} electrons (even). Assuming closed-shell (spin=0).")
    scf_method = scf.RHF # Use Restricted HF for closed-shell
    dft_method = dft.RKS # Use Restricted KS-DFT for closed-shell
else:
    mol_spin = 1  # Odd number of electrons -> Doublet (2S = 1)
    print(f"Calculated {num_electrons} electrons (odd). Assuming open-shell (spin=1).")
    scf_method = scf.UHF # Use Unrestricted HF for open-shell
    dft_method = dft.UKS # Use Unrestricted KS-DFT for open-shell

Calculated 39 electrons (odd). Assuming open-shell (spin=1).


In [37]:
# --- 3. Define the molecule using pyscf.gto.M ---
mol = gto.M(
    atom = atom_string,
    basis = '6-31g*',
    charge = desired_molecular_charge,
    spin = mol_spin,
    unit = 'Angstrom'
)
# PySCF automatically checks consistency here and will raise the RuntimeError
# if mol.nelectron (PySCF's own count) and mol.spin are inconsistent.
# We've preemptively set spin based on our calculated num_electrons.
# If mol.nelectron (from PySCF) differs from our num_electrons,
# it indicates a mismatch in atom symbols or a hidden issue.
print(f"PySCF's determined number of electrons: {mol.nelectron}")

PySCF's determined number of electrons: 39


In [38]:
# --- 4. Perform the calculation ---

# --- Option A: Hartree-Fock ---
print(f"--- Running {scf_method.__name__} calculation ---")
mf_scf = scf_method(mol)
mf_scf.verbose = 4
scf_energy = mf_scf.kernel()

if scf_energy is not None:
    print(f"\n{scf_method.__name__} Calculation successful!")
    print(f"{scf_method.__name__} Energy: {scf_energy:.6f} Hatrees")
    print(f"{scf_method.__name__} Energy: {scf_energy * 627.509:.3f} kcal/mol\n")
else:
    print(f"{scf_method.__name__} calculation did not converge.\n")

--- Running UHF calculation ---


******** <class 'pyscf.scf.uhf.UHF'> ********
method = UHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /tmp/tmpg7v_108i
max_memory 4000 MB (current use 843 MB)
number electrons alpha = 20  beta = 19
Set gradient conv threshold to 3.16228e-05
init E= -549.059116187159
  alpha nocc = 20  HOMO = -0.257757959992695  LUMO = -0.171217048580926
  beta  nocc = 19  HOMO = -0.270756239097285  LUMO = -0.257757959992695

WARN: system HOMO -0.257757959992695 >= system LUMO -0.257757959992695

cycle= 1 E= -547.957150699483  delta_E=  1.1  |g|= 1.08  |ddm|= 1.66
  alpha nocc = 20  HOMO = -0.118941930934589  LUMO = -0.0971009469944952
  beta  nocc = 19  HOMO = -0.138179299231968  LUMO = -0.112856285798699
cycle= 2 E= -540.052431

In [39]:
# --- Option B: Density Functional Theory (DFT) ---
functional = 'B3LYP'
print(f"--- Running {dft_method.__name__} ({functional}) calculation ---")
mf_dft = dft_method(mol)
mf_dft.xc = functional
mf_dft.verbose = 4
dft_energy = mf_dft.kernel()

if dft_energy is not None:
    print(f"\n{functional} {dft_method.__name__} Calculation successful!")
    print(f"{functional} {dft_method.__name__} Energy: {dft_energy:.6f} Hatrees")
    print(f"{functional} {dft_method.__name__} Energy: {dft_energy * 627.509:.3f} kcal/mol\n")
else:
    print(f"{functional} {dft_method.__name__} calculation did not converge.\n")

--- Running UKS (B3LYP) calculation ---


******** <class 'pyscf.dft.uks.UKS'> ********
method = UKS
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /tmp/tmp_zrq7n0t
max_memory 4000 MB (current use 853 MB)
number electrons alpha = 20  beta = 19
XC library pyscf.dft.libxc version 7.0.0
    S. Lehtola, C. Steigemann, M. J.T. Oliveira, and M. A.L. Marques.,  SoftwareX 7, 1–5 (2018)
XC functionals = B3LYP
    P. J. Stephens, F. J. Devlin, C. F. Chabalowski, and M. J. Frisch.,  J. Phys. Chem. 98, 11623 (1994)
small_rho_cutoff = 1e-07
Set gradient conv threshold to 3.16228e-05


/home/kyle/miniforge3/envs/vina_vqe/lib/python3.13/site-packages/pyscf/dft/libxc.py:511: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


init E= -550.952194482401
  alpha nocc = 20  HOMO = -0.265527724730422  LUMO = -0.233706469298888
  beta  nocc = 19  HOMO = -0.271662122881797  LUMO = -0.265527724730422

WARN: system HOMO -0.265527724730422 >= system LUMO -0.265527724730422

cycle= 1 E= -549.561473211971  delta_E= 1.39  |g|= 1.17  |ddm|= 1.75
  alpha nocc = 20  HOMO = -0.19179191709853  LUMO = -0.0946247395714075
  beta  nocc = 19  HOMO = -0.239485851202449  LUMO = -0.187110075320403
cycle= 2 E= -530.949385785489  delta_E= 18.6  |g|= 2.86  |ddm|= 8.29
  alpha nocc = 20  HOMO = -0.064413986963929  LUMO = -0.0538725876270919
  beta  nocc = 19  HOMO = -0.1844355233096  LUMO = -0.0635351124953446

WARN: system HOMO -0.0635351124953446 >= system LUMO -0.0635351124953446

cycle= 3 E= -546.499590264561  delta_E= -15.6  |g|= 1.62  |ddm|=  7.9
  alpha nocc = 20  HOMO = -0.211931974172019  LUMO = -0.192038350570095
  beta  nocc = 19  HOMO = -0.232776968250258  LUMO = -0.188025287637766
cycle= 4 E= -549.475272972685  delta_E= -2

### Quantum Computing Benchmark (VQE)

In [58]:
driver = PySCFDriver(
    atom=atom_string, 
    basis="sto3g",
    charge=desired_molecular_charge,
    spin=mol_spin, # <--- CRITICAL: Pass the calculated spin
)
problem = driver.run()

In [59]:
print(f"\nInitial (full space) number of spatial orbitals: {problem.num_spatial_orbitals}")
print(f"Initial (full space) number of alpha electrons: {problem.num_alpha}")
print(f"Initial (full space) number of beta electrons: {problem.num_beta}")


Initial (full space) number of spatial orbitals: 27
Initial (full space) number of alpha electrons: 20
Initial (full space) number of beta electrons: 19


In [60]:
active_electrons_to_keep = (10, 9) # Total 11 active electrons (6 alpha, 5 beta)
                                   # (47 - 11 = 36 inactive electrons, which is EVEN)
active_orbitals_to_keep = 10       
transformer = ActiveSpaceTransformer(
    num_electrons=active_electrons_to_keep, # Total electrons in the active space (alpha + beta)
    num_spatial_orbitals=active_orbitals_to_keep     # Total spatial orbitals in the active space
)
problem = transformer.transform(problem)

In [61]:
mapper = JordanWignerMapper()
qubit_op = mapper.map(problem.second_q_ops()[0])
print(f"\nQubit Hamiltonian has {qubit_op.num_qubits} qubits.")


Qubit Hamiltonian has 20 qubits.


In [62]:
qubit_op

SparsePauliOp(['IIIIIIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIZ', 'IIIIIIIIIIIIIIIIIIZI', 'IIIIIIIIIIIIIIIIIIZZ', 'IIIIIIIIIIIIIIIIIYYI', 'IIIIIIIIIIIIIIIIIYYZ', 'IIIIIIIIIIIIIIIIIXXI', 'IIIIIIIIIIIIIIIIIXXZ', 'IIIIIIIIIIIIIIIIYZYI', 'IIIIIIIIIIIIIIIIYZYZ', 'IIIIIIIIIIIIIIIIXZXI', 'IIIIIIIIIIIIIIIIXZXZ', 'IIIIIIIIIIIIIIIYZZYI', 'IIIIIIIIIIIIIIIYZZYZ', 'IIIIIIIIIIIIIIIXZZXI', 'IIIIIIIIIIIIIIIXZZXZ', 'IIIIIIIIIIIIIIYZZZYI', 'IIIIIIIIIIIIIIYZZZYZ', 'IIIIIIIIIIIIIIXZZZXI', 'IIIIIIIIIIIIIIXZZZXZ', 'IIIIIIIIIIIIIYZZZZYI', 'IIIIIIIIIIIIIYZZZZYZ', 'IIIIIIIIIIIIIXZZZZXI', 'IIIIIIIIIIIIIXZZZZXZ', 'IIIIIIIIIIIIYZZZZZYI', 'IIIIIIIIIIIIYZZZZZYZ', 'IIIIIIIIIIIIXZZZZZXI', 'IIIIIIIIIIIIXZZZZZXZ', 'IIIIIIIIIIIYZZZZZZYI', 'IIIIIIIIIIIYZZZZZZYZ', 'IIIIIIIIIIIXZZZZZZXI', 'IIIIIIIIIIIXZZZZZZXZ', 'IIIIIIIIIIYZZZZZZZYI', 'IIIIIIIIIIYZZZZZZZYZ', 'IIIIIIIIIIXZZZZZZZXI', 'IIIIIIIIIIXZZZZZZZXZ', 'IIIIIIIIIIIIIIIIIZII', 'IIIIIIIIIIIIIIIIIZIZ', 'IIIIIIIIIIIIIIIIYYII', 'IIIIIIIIIIIIIIIIYYIZ', 'IIIIIIIIIIIIIIIIXXII', '